# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 파인튜닝 해 봅시다.

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

## 데이터셋: KoAlpaca v1.1a

In [ ]:
from datasets import load_dataset

my_data = load_dataset("json", data_files="our_data_input.json")
# data = load_dataset("beomi/KoAlpaca-v1.1a")

In [ ]:
my_data

In [ ]:
# # data
my_data = my_data.map(
    lambda x:
    {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
    if x['input'] else
    {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
)

## 쪼개진 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 터지지 않습니다.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

## 텍스트 데이터만 tokenize

In [ ]:
my_data = my_data.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [ ]:
print(len(my_data))

In [ ]:
my_data['train'][0]['text']

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
!nvidia-smi

## 학습하기!

- 이번 예제에서는 22k개의 아주아주 일부분인 100개 스텝만 학습해봅시다.

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=my_data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8, # base 2
        gradient_accumulation_steps=1,
        # max_steps=50, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=2500,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
# trainer.train("outputs/checkpoint-11000") # 이어서 학습할 경우


In [ ]:
model.save_pretrained('./')

In [ ]:
print("wow")

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

In [ ]:
gen('사회보장정보원은 뭐하는 곳이야?')

## 모델 저장 & 업로드

In [ ]:
!huggingface-cli login

In [ ]:
# 아래 아이디 부분을 수정해서 쓰세요.
# model.push_to_hub('허깅페이스아이디/qlora-koalpaca-polyglot-12.8b-50step')
model.push_to_hub('Noveled/ssis-chat')